<a href="https://colab.research.google.com/github/akash-singh-10/Deep-Learning/blob/main/DLA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
import re

In [30]:
data = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,|
we conjure the spirits of the computer with our spells."""

In [31]:
sentences = data.split(".")
sentences

['We are about to study the idea of a computational process',
 '\nComputational processes are abstract beings that inhabit computers',
 '\nAs they evolve, processes manipulate other abstract things called data',
 '\nThe evolution of a process is directed by a pattern of rules\ncalled a program',
 ' People create programs to direct processes',
 ' In effect,|\nwe conjure the spirits of the computer with our spells',
 '']

In [32]:
clean = []
for sentence in sentences:
  if sentence == "":
    continue;
  sentence = re.sub('[^A-Za-z0-9]+',' ', sentence)
  sentence = re.sub(r'(?:^| )\w(?:$| )',' ', sentence).strip()
  sentence = sentence.lower()
  clean.append(sentence)
clean

['we are about to study the idea of computational process',
 'computational processes are abstract beings that inhabit computers',
 'as they evolve processes manipulate other abstract things called data',
 'the evolution of process is directed by pattern of rules called program',
 'people create programs to direct processes',
 'in effect we conjure the spirits of the computer with our spells']

In [33]:
collec = clean

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(collec)
seq = tokenizer.texts_to_sequences(collec)
print(seq)

[[4, 5, 11, 6, 12, 1, 13, 2, 7, 8], [7, 3, 5, 9, 14, 15, 16, 17], [18, 19, 20, 3, 21, 22, 9, 23, 10, 24], [1, 25, 2, 8, 26, 27, 28, 29, 2, 30, 10, 31], [32, 33, 34, 6, 35, 3], [36, 37, 4, 38, 1, 39, 2, 1, 40, 41, 42, 43]]


In [35]:
i2word = {}
word2i = {}
for ind1, sequence in enumerate(seq):
  print(sequence)
  words = clean[ind1].split()
  print(words)
  for ind2, val in enumerate(sequence):
    i2word[val] = words[ind2]
    word2i[words[ind2]] = val

print(i2word)
print(word2i)

[4, 5, 11, 6, 12, 1, 13, 2, 7, 8]
['we', 'are', 'about', 'to', 'study', 'the', 'idea', 'of', 'computational', 'process']
[7, 3, 5, 9, 14, 15, 16, 17]
['computational', 'processes', 'are', 'abstract', 'beings', 'that', 'inhabit', 'computers']
[18, 19, 20, 3, 21, 22, 9, 23, 10, 24]
['as', 'they', 'evolve', 'processes', 'manipulate', 'other', 'abstract', 'things', 'called', 'data']
[1, 25, 2, 8, 26, 27, 28, 29, 2, 30, 10, 31]
['the', 'evolution', 'of', 'process', 'is', 'directed', 'by', 'pattern', 'of', 'rules', 'called', 'program']
[32, 33, 34, 6, 35, 3]
['people', 'create', 'programs', 'to', 'direct', 'processes']
[36, 37, 4, 38, 1, 39, 2, 1, 40, 41, 42, 43]
['in', 'effect', 'we', 'conjure', 'the', 'spirits', 'of', 'the', 'computer', 'with', 'our', 'spells']
{4: 'we', 5: 'are', 11: 'about', 6: 'to', 12: 'study', 1: 'the', 13: 'idea', 2: 'of', 7: 'computational', 8: 'process', 3: 'processes', 9: 'abstract', 14: 'beings', 15: 'that', 16: 'inhabit', 17: 'computers', 18: 'as', 19: 'they', 2

In [36]:
vocab_size = len(tokenizer.word_index)+1
embedding_size = 10
window = 2

contexts = []
targets = []

for s in seq:
  for i in range(window, len(s)-window):
    context = s[i-window:i] + s[i+1 : i+1+window]
    target = s[i]
    contexts.append(context)
    targets.append(target)

for i in range(5):
  words=[]
  target = i2word.get(targets[i])
  for j in contexts[i]:
    words.append(i2word.get(j))
  print(words, "->", target)


['we', 'are', 'to', 'study'] -> about
['are', 'about', 'study', 'the'] -> to
['about', 'to', 'the', 'idea'] -> study
['to', 'study', 'idea', 'of'] -> the
['study', 'the', 'of', 'computational'] -> idea


In [37]:
X = np.array(contexts)
Y = np.array(targets)

In [38]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim=embedding_size, input_length=2*window))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(units=vocab_size, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 10)             440       
                                                                 
 lambda_1 (Lambda)           (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 256)               2816      
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 dense_5 (Dense)             (None, 44)                22572     
                                                                 
Total params: 157412 (614.89 KB)
Trainable params: 157412 (614.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X,Y,epochs=200, verbose=1)

Epoch 1/200
2/2 [==============================] - 1s 12ms/step - loss: 3.7845 - accuracy: 0.0294
Epoch 2/200
2/2 [==============================] - 0s 13ms/step - loss: 3.7754 - accuracy: 0.1176
Epoch 3/200
2/2 [==============================] - 0s 12ms/step - loss: 3.7686 - accuracy: 0.1176
Epoch 4/200
2/2 [==============================] - 0s 12ms/step - loss: 3.7610 - accuracy: 0.1176
Epoch 5/200
2/2 [==============================] - 0s 13ms/step - loss: 3.7516 - accuracy: 0.1176
Epoch 6/200
2/2 [==============================] - 0s 13ms/step - loss: 3.7407 - accuracy: 0.1176
Epoch 7/200
2/2 [==============================] - 0s 19ms/step - loss: 3.7260 - accuracy: 0.1176
Epoch 8/200
2/2 [==============================] - 0s 13ms/step - loss: 3.7097 - accuracy: 0.1176
Epoch 9/200
2/2 [==============================] - 0s 13ms/step - loss: 3.6886 - accuracy: 0.1176
Epoch 10/200
2/2 [==============================] - 0s 12ms/step - loss: 3.6636 - accuracy: 0.1176
Epoch 11/200
2/2 [=

In [40]:
test = [
    "we are to study",
    "create programs direct processes",
    "spirits process study program",
    "idea study people create"
]

In [41]:
for t in test:
  words = t.split(" ")
  x_test = []
  for i in words:
    x_test.append(word2i.get(i))
  x_test = np.array([x_test])
  print(words)
  pred = model.predict(x_test)
  print(i2word.get(np.argmax(pred[0])))

['we', 'are', 'to', 'study']
1/1 [==============================] - 0s 96ms/step
about
['create', 'programs', 'direct', 'processes']
1/1 [==============================] - 0s 24ms/step
to
['spirits', 'process', 'study', 'program']
1/1 [==============================] - 0s 43ms/step
the
['idea', 'study', 'people', 'create']
1/1 [==============================] - 0s 33ms/step
programs
